In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df['월'] = df['발생일시'].str[5:7]

In [ ]:
df.head()

In [ ]:
df['월'] = df['월'].astype(int)

# 월을 기반으로 계절을 반환하는 함수 정의
def get_season(month):
    if month in [12, 1, 2]:
        return '겨울'
    elif month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'

# '계절' 컬럼 생성
df['계절'] = df['월'].apply(get_season)

In [ ]:
df

In [ ]:

# '기온' 컬럼에서 '℃' 제거 후, pd.to_numeric()을 사용하여 float형으로 변환 (오류 발생 시 NaN 처리)
df['기온'] = pd.to_numeric(df['기온'].str.replace('℃', ''), errors='coerce')

print(df)


In [ ]:
df['기온'] = df['기온'].apply(lambda x: x / 10 if x >= 100 else x)

In [ ]:
df.groupby('월')['기온'].mean()

In [ ]:
# 월별 평균 기온 계산 후 각 행에 해당 월의 평균 기온을 "평균기온" 컬럼에 저장
df['평균기온'] = df.groupby('월')['기온'].transform('mean')


In [ ]:
df[df['계절'] == '여름']

In [ ]:
df.columns

In [ ]:
df = df.drop(columns='평균 기온')

In [ ]:
df_s = df[df['계절'] == '여름']

In [ ]:
df_s[100:150]

In [ ]:
# "위치" 컬럼 생성: 기온이 평균기온보다 높으면 '아래', 그렇지 않으면 '위'
df['위치'] = df.apply(lambda row: '아래' if row['기온'] > row['평균기온'] else '위', axis=1)


In [ ]:
df.to_csv("df.csv")

In [ ]:
df['위치'].value_counts()

In [ ]:
df[['위치','재발방지대책 및 향후조치계획']]


In [ ]:
df_up  = df[df['위치'] == '위']
df_down  = df[df['위치'] == '아래']


In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [ ]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
grouped_up = df_up.groupby("인적사고")
grouped_down = df_down.groupby("인적사고")


In [ ]:
res = {}
cosine_res = []
for name, group in tqdm(grouped_up):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

In [ ]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
res_down = {}
cosine_res_down = []
for name, group in tqdm(grouped_down):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res_down += similarity[similarity.mean(axis=1).argmax()].tolist()
    res_down[name] = plan.iloc[similarity.mean(axis=1).argmax()]

In [ ]:
arr = cosine_res_down

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
res

In [ ]:
res_down